# Using CARTOframes to view earthquakes from the last 30 days

In [ ]:
import cartoframes
from cartoframes import Credentials
from cartoframes import Layer, BaseMap, styling
import pandas as pd
%matplotlib inline

In [ ]:
USERNAME = 'michellemho-carto'  # <-- replace with your username 
APIKEY = 'abcde'       # <-- your CARTO API key
creds = Credentials(username=USERNAME, 
                    key=APIKEY)
creds.save()  # save credentials for later use (not dependent on Python session)
cc = cartoframes.CartoContext(creds)

In [ ]:
cc = cartoframes.CartoContext()

In [ ]:
# Use Pandas to read a CSV from the USGS feed of earthquakes from the last 30 days
earthquakes = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv')
earthquakes.head()

In [ ]:
# Write earthquake data to CARTO
cc.write(earthquakes, 'earthquakes_30', lnglat=('longitude','latitude'), overwrite=True)

In [ ]:
# Map the earthquake data
cc.map(layers=[Layer('earthquakes_30')])

In [ ]:
# Style the color and size by the magnitude
cc.map(layers=[Layer('earthquakes_30',
       color={'column': 'mag', 'scheme': styling.redOr(5, bin_method='equal')},
       size={'column':'mag', 'bins':5, 'bin_method':'equal'})])

In [ ]:
# Try torque animation of time-stamp column

# Make sure time-stamp column is a pandas date type
earthquakes['time'] = pd.to_datetime(earthquakes.time)

In [ ]:
earthquakes.head(2)

In [ ]:
# Update CARTO dataset
cc.write(earthquakes, 'earthquakes_30', lnglat=('longitude','latitude'), overwrite=True)

In [ ]:
# Create an animated map by specifying the time of the layer
cc.map(layers=[Layer('earthquakes_30', time = {'column':'time', 'method':'sum', 'trails':4, 'duration':30})])